#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [1]:
import os, sys 

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

# from src import __version__
# print(__version__)

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [2]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [3]:
yf.__version__

'1.1.0'

In [4]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1 * 365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [23]:

import importlib
import get_stocks_data as gs
importlib.reload(gs)

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["ESSAR?SHPNG.NS","LATENTVIEW.NS","BHEL.NS"]
# tickers = tickers[:500] # for testing, limit to first 10 tickers

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2026-02-06 10:53:03,376 - INFO - 📈 Fetching data for 2178 tickers...
2026-02-06 10:53:03,378 - INFO - Fetching data from 2025-01-07 to 2026-02-06


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[                       1%                       ]  29 of 2178 completed2026-02-06 10:53:05,033 - ERROR - $DHANI.NS: possibly delisted; no price data found  (1d 2025-01-07 10:53:03.377787 -> 2026-02-06 10:53:03.377787) (Yahoo error = "No data found, symbol may be delisted")
[**                     4%                       ]  85 of 2178 completed2026-02-06 10:53:07,700 - ERROR - $SMSLIFE.NS: possibly delisted; no price data found  (1d 2025-01-07 10:53:03.377787 -> 2026-02-06 10:53:03.377787) (Yahoo error = "No data found, symbol may be delisted")
[*******               14%                       ]  307 of 2178 completed2026-02-06 10:53:15,968 - ERROR - $TATAMOTORS.NS: possibly delisted; no price data found  (1d 2025-01-07 10:53:03.377787 -> 2026-02-06 10:53:03.377787) (Yahoo error = "No data found, symbol may be delisted")
[********************* 43%                       ]  929 of 2178 completed2026-02-06 10:53:39,482 - ERROR - $IEL.NS: possibly delisted; no price data found  (1d 2025-01

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2025-01-07,20MICRONS.NS,NaN,249.005447,253.753991,238.174358,240.911994,83202.0,249.005447,249.005447,249.005447,249.005447,NaN,NaN,20MICRONS
1,2025-01-08,20MICRONS.NS,NaN,244.366394,249.373778,238.920985,249.373778,41548.0,248.563633,248.823524,248.913585,248.959288,NaN,NaN,20MICRONS
2,2025-01-09,20MICRONS.NS,NaN,239.488419,247.681413,235.436710,243.888545,27429.0,247.699327,248.457441,248.726948,248.865050,NaN,NaN,20MICRONS
3,2025-01-10,20MICRONS.NS,NaN,234.222198,240.911987,229.961442,239.488424,40503.0,246.415791,247.899196,248.439725,248.719350,NaN,NaN,20MICRONS
4,2025-01-13,20MICRONS.NS,NaN,223.341339,243.689429,219.110447,238.901057,172734.0,244.218224,246.936143,247.942727,248.466833,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [ ]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)
# print("Total accumulation signals detected:", len(accumulation_df))

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)
# print("Total accumulation signals entry:", len(accum_df))


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

Total accumulation signals detected: 2165
         Symbol       Date    AD_Slope  Above_20EMA_%  Avg_Volume_Spike  \
1794  SOLARINDS 2026-02-05  11881.5675           80.0              1.06   
1700  SEAMECLTD 2026-02-06  22399.2192           90.0              1.24   
1389  ORICONENT 2026-02-05  45550.6274           95.0              1.37   
321      BORANA 2026-02-06  26141.8666           95.0              1.40   
363     CEIGALL 2026-02-06  46113.7445           90.0              0.78   

        RSI  Accumulating         20ema         50ema        200ema  \
1794  55.78          True  13132.513014  13061.740593  13200.138864   
1700  76.56          True   1175.424935   1089.655993    985.174382   
1389  64.56          True     63.549288     61.033841     53.417080   
321   72.09          True    348.725911    316.112522    268.263816   
363   63.37          True    275.367501    265.807298    265.852770   

             Close            7dv           30dv  
1794  13384.000000  242101.00

,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv


#### EMA Signal indicator 


In [ ]:
import importlib
import ema_signal as emasignal
importlib.reload(emasignal)

# Detect Golden/Death cross with volume confirmation

long_stocks, short_stocks, ema_df = emasignal.generate_ema_signals(
    stock_df,
    analysis_period=15,
    vol_thresh=1.5,
    output_period=30
)
long_stocks.head()


status  = emasignal.save_EMA_signal_to_db(long_stocks, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
ema_df.head()

153 tickers with Close/200ema crossover between 2026-01-22–2026-02-06
32 tickers with 50ema/200ema crossover between 2026-01-22–2026-02-06
✅ Metadata fetched for 179 total stocks
✅ Final signals: 161 long | 98 short
2
✅ Saved 161 new EMA signal records to SIGNAL_EMA_CROSS
📦 Table now has 4200 total records.
✅ Database updated successfully


,Symbol,signal_1,Date_1,Price_1,7dratio_1,30dratio_1,Vol1,Mom1,signal_2,Date_2,Price_2,7dratio_2,30dratio_2,Vol2,Mom2,currentPrice,Capitalization,MCap,upFrom52wlow,PE,PB,PBXPE,debtToEquity,PS,PricetoCash,EPS,CashperShare,update date,beta,AD,AD_50
0,360ONE,Price_gt_200ema,2026-01-23,1112.0,0.66,2.05,2.05,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1103.0,Large Cap,44758.0,0.65,39.0,5.0,185.0,1.47,10.58,34.922595,29.0,32.0,2026-02-06,0.26,NaN,NaN
1,AADHARHFC,Price_gt_200ema,2026-01-30,491.0,2.24,1.24,2.24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,474.0,Large Cap,20558.0,0.60,20.0,3.0,60.0,2.55,9.59,11.4535,23.0,41.0,2026-02-06,0.00,NaN,NaN
2,AARTIIND,Price_gt_200ema,2026-02-03,430.0,5.37,2.71,5.37,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441.0,Mid Cap,16031.0,0.66,42.0,3.0,120.0,0.70,2.00,94.670813,10.0,5.0,2026-02-06,0.38,NaN,NaN
4,ABB,Price_gt_200ema,2026-01-29,5474.0,4.24,2.27,4.24,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5740.0,Large Cap,121632.0,0.68,69.0,17.0,1160.0,0.01,9.35,23.105785,84.0,248.0,2026-02-06,0.47,NaN,NaN
5,ABSLAMC,Price_gt_200ema,2026-01-23,780.0,3.47,0.97,3.47,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,805.0,Large Cap,23269.0,0.71,23.0,7.0,149.0,0.02,10.92,204.316912,35.0,4.0,2026-02-06,0.18,NaN,NaN


##### Update the Stock returns table

In [20]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 2291
✅ Total signals after merging: 84578
📦 Existing SIGNAL_RETURNS records: 2813
✅ New unique records found: 9
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
189,ASTRAMICRO,Golden cross,2026-01-28,962.0,4,-1.0,952.349976,2026-02-04,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-2.71,935.950012,2026-02-05,-1.0,<NA>,<NA>,<NA>,<NA>,<NA>,-2.71
268,BBOX,Golden cross,2026-02-03,541.0,4,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-7.43,500.799988,2026-02-06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-7.43
474,DATAPATTNS,Death cross,2026-02-05,2524.0,2,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,5.07,2652.000000,2026-02-06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5.07
475,DATAPATTNS,Death cross,2026-02-02,2558.0,2,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,3.67,2652.000000,2026-02-06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3.67
476,DATAPATTNS,Death cross,2026-01-29,2562.0,1,-1.5,2523.500000,2026-02-05,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,3.51,2652.000000,2026-02-06,-1.5,<NA>,<NA>,<NA>,<NA>,<NA>,3.51
